In [ ]:
import torch
print(torch.cuda.is_available())  # Should print 'True' if GPU is available
print(torch.cuda.get_device_name(0))  # Prints the name of the GPU

True
Tesla T4


In [ ]:
# Import necessary libraries
import snntorch as snn
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.float

# Define transformations for GTSRB dataset
transform = transforms.Compose([
    transforms.Resize((32, 32)),  # Resize to match GTSRB dimensions
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1), # Data augmentation
    transforms.RandomRotation(10),  # Data augmentation
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # Adjust normalization for GTSRB if necessary
])

# Load GTSRB dataset from torchvision
data_path = './data/gtsrb'  # Set your data path
train_data = datasets.GTSRB(root=data_path, split='train', download=False, transform=transform)
test_data = datasets.GTSRB(root=data_path, split='test', download=False, transform=transform)
train_loader = DataLoader(train_data, batch_size=64, shuffle=True, drop_last=True)  # Smaller batch size
test_loader = DataLoader(test_data, batch_size=64, shuffle=True, drop_last=True)

# Update model parameters for GTSRB
num_inputs = 32 * 32 * 3  # Account for RGB channels
num_hidden1 = 1200
num_hidden2 = 800
num_outputs = 43      # Number of classes in GTSRB

# Temporal Dynamics
num_steps = 50  # Increased for more temporal processing
beta = 0.9

# Define Network
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        # Initialize layers
        self.fc1 = nn.Linear(num_inputs, num_hidden1)
        self.lif1 = snn.Leaky(beta=beta)
        self.fc2 = nn.Linear(num_hidden1, num_hidden2)
        self.lif2 = snn.Leaky(beta=beta)
        self.fc3 = nn.Linear(num_hidden2, num_outputs)
        self.lif3 = snn.Leaky(beta=beta)

    def forward(self, x):
        # Initialize hidden states at t=0
        mem1 = self.lif1.init_leaky()
        mem2 = self.lif2.init_leaky()
        mem3 = self.lif3.init_leaky()

        # Record the final layer
        spk3_rec = []
        mem3_rec = []

        # Time-loop
        for step in range(num_steps):
            cur1 = self.fc1(x.flatten(1))  # Flatten to batch x (32*32*3)
            spk1, mem1 = self.lif1(cur1, mem1)
            cur2 = self.fc2(spk1)
            spk2, mem2 = self.lif2(cur2, mem2)
            cur3 = self.fc3(spk2)
            spk3, mem3 = self.lif3(cur3, mem3)

            # Store results
            spk3_rec.append(spk3)
            mem3_rec.append(mem3)

        return torch.stack(spk3_rec, dim=0), torch.stack(mem3_rec, dim=0)  # time-steps x batch x num_outputs

# Initialize network, loss function, and optimizer
net = Net().to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=1e-4)  # Decreased learning rate

# Training parameters
num_epochs = 10  # Increased epochs
counter = 0

# Outer training loop
for epoch in range(num_epochs):
    net.train()
    for data, targets in train_loader:
        data, targets = data.to(device), targets.to(device)

        # Forward pass
        spk_rec, _ = net(data)

        # Compute loss over time
        loss_val = loss_fn(spk_rec.sum(0), targets)  # Sum spikes over time, then apply CrossEntropy

        # Backpropagation and optimization
        optimizer.zero_grad()
        loss_val.backward()
        optimizer.step()

        # Print loss
        if counter % 100 == 0:
            print(f"Epoch: {epoch} \t Iteration: {counter} \t Train Loss: {loss_val.item()}")
        counter += 1

# Testing the model
net.eval()
correct = 0
total = 0

with torch.no_grad():
    for data, targets in test_loader:
        data, targets = data.to(device), targets.to(device)

        # Forward pass through the network
        spk_rec, _ = net(data)

        # Sum the spikes over time and compute predictions
        output_sum = spk_rec.sum(0)  # Sum over time
        _, predicted = torch.max(output_sum, 1)

        # Update correct predictions and total samples
        correct += (predicted == targets).sum().item()
        total += targets.size(0)

# Calculate accuracy
accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")

100%|██████████| 187M/187M [00:08<00:00, 22.3MB/s]


Extracting data/gtsrb/gtsrb/GTSRB-Training_fixed.zip to data/gtsrb/gtsrb


100%|██████████| 89.0M/89.0M [00:04<00:00, 19.6MB/s]


Extracting data/gtsrb/gtsrb/GTSRB_Final_Test_Images.zip to data/gtsrb/gtsrb


100%|██████████| 99.6k/99.6k [00:00<00:00, 221kB/s]


Extracting data/gtsrb/gtsrb/GTSRB_Final_Test_GT.zip to data/gtsrb/gtsrb
Epoch: 0 	 Iteration: 0 	 Train Loss: 3.7434420585632324
Epoch: 0 	 Iteration: 100 	 Train Loss: 2.390298366546631
Epoch: 0 	 Iteration: 200 	 Train Loss: 2.4585907459259033
Epoch: 0 	 Iteration: 300 	 Train Loss: 1.5240609645843506
Epoch: 0 	 Iteration: 400 	 Train Loss: 1.3771090507507324
Epoch: 1 	 Iteration: 500 	 Train Loss: 1.2007291316986084
Epoch: 1 	 Iteration: 600 	 Train Loss: 0.9711872935295105
Epoch: 1 	 Iteration: 700 	 Train Loss: 0.8612615466117859
Epoch: 1 	 Iteration: 800 	 Train Loss: 0.8232224583625793
Epoch: 2 	 Iteration: 900 	 Train Loss: 0.8351987600326538
Epoch: 2 	 Iteration: 1000 	 Train Loss: 0.5061237215995789
Epoch: 2 	 Iteration: 1100 	 Train Loss: 0.6144967675209045
Epoch: 2 	 Iteration: 1200 	 Train Loss: 0.555315375328064
Epoch: 3 	 Iteration: 1300 	 Train Loss: 0.5759874582290649
Epoch: 3 	 Iteration: 1400 	 Train Loss: 0.6851558089256287
Epoch: 3 	 Iteration: 1500 	 Train Loss: 0

In [7]:
import snntorch as snn
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import snntorch.functional as SF
from torchvision.datasets import ImageFolder

# Define Constants
beta = 0.9
num_steps = 50  # Number of time steps
num_classes = 43
batch_size = 64
dtype = torch.float
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define Transformations for GTSRB with Data Augmentation
transform = transforms.Compose([
    transforms.Resize((32, 32)),                       # Resize images to 32x32 pixels
    transforms.ColorJitter(brightness=0.2,             # Augment brightness, contrast, etc.
                           contrast=0.2,
                           saturation=0.2,
                           hue=0.1),
    transforms.RandomRotation(10),                     # Random rotation up to 10 degrees
    transforms.ToTensor(),                             # Convert images to PyTorch tensors
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize
])

# Load GTSRB dataset from torchvision
data_path = './data/gtsrb'  # Set your data path
train_data = datasets.GTSRB(root=data_path, split='train', download=False, transform=transform)
test_data = datasets.GTSRB(root=data_path, split='test', download=False, transform=transform)
train_loader = DataLoader(train_data, batch_size=64, shuffle=True, drop_last=True)  # Smaller batch size
test_loader = DataLoader(test_data, batch_size=64, shuffle=True, drop_last=True)

# Define Spiking CNN Network
class ConvNet(nn.Module):
    def __init__(self):
        super().__init__()

        # Convolutional and Pooling Layers with Leaky Integrate-and-Fire (LIF) Neurons
        self.conv1 = nn.Conv2d(3, 8, kernel_size=5, padding="same")  # 3 input channels for RGB
        self.lif1 = snn.Leaky(beta=beta)
        self.mp1 = nn.MaxPool2d(2)

        self.conv2 = nn.Conv2d(8, 24, kernel_size=5, padding="same")
        self.lif2 = snn.Leaky(beta=beta)
        self.mp2 = nn.MaxPool2d(2)

        # Fully Connected Layer
        self.fc = nn.Linear(24 * 8 * 8, num_classes)  # Adjusted for 32x32 input, after pooling
        self.lif3 = snn.Leaky(beta=beta)

    def forward(self, x):
        # Initialize hidden states at t=0 for each LIF layer
        mem1 = self.lif1.init_leaky()
        mem2 = self.lif2.init_leaky()
        mem3 = self.lif3.init_leaky()

        # Record the final layer
        spk3_rec = []
        mem3_rec = []

        # Time-step loop
        for step in range(num_steps):
            cur1 = self.conv1(x)
            spk1, mem1 = self.lif1(self.mp1(cur1), mem1)
            cur2 = self.conv2(spk1)
            spk2, mem2 = self.lif2(self.mp2(cur2), mem2)
            cur3 = self.fc(spk2.flatten(1))
            spk3, mem3 = self.lif3(cur3, mem3)

            spk3_rec.append(spk3)
            mem3_rec.append(mem3)

        return torch.stack(spk3_rec, dim=0), torch.stack(mem3_rec, dim=0)

# Initialize Network
convnet = ConvNet().to(device)

# Loss and Optimizer
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(convnet.parameters(), lr=1e-3)

# Training Loop
num_epochs = 10
loss_hist = []
counter = 0

for epoch in range(num_epochs):
    for data, targets in train_loader:
        data = data.to(device)
        targets = targets.to(device)

        # Forward pass
        convnet.train()
        spk_rec, _ = convnet(data)

        # Calculate loss and perform optimization
        loss_val = loss(spk_rec.sum(0), targets)  # Sum spikes over time
        optimizer.zero_grad()
        loss_val.backward()
        optimizer.step()

        loss_hist.append(loss_val.item())

        # Print progress
        if counter % 100 == 0:
            print(f"Iteration: {counter}  Train Loss: {loss_val.item()}")
        counter += 1

# Accuracy Measurement Function
def measure_accuracy(model, dataloader):
    with torch.no_grad():
        model.eval()
        correct = 0
        total = 0

        for data, targets in dataloader:
            data = data.to(device)
            targets = targets.to(device)

            # Forward pass
            spk_rec, _ = model(data)
            spike_count = spk_rec.sum(0)
            _, predicted = spike_count.max(1)

            correct += (predicted == targets).sum().item()
            total += targets.size(0)

    return correct / total

# Evaluate Network
accuracy = measure_accuracy(convnet, test_loader)
print(f"ConvNet Accuracy on GTSRB with Augmentation: {accuracy * 100:.2f}%")


Iteration: 0  Train Loss: 7.057320594787598
Iteration: 100  Train Loss: 2.316955089569092
Iteration: 200  Train Loss: 1.8013885021209717
Iteration: 300  Train Loss: 0.5345782041549683
Iteration: 400  Train Loss: 0.5596723556518555
Iteration: 500  Train Loss: 0.8023688793182373
Iteration: 600  Train Loss: 0.38728249073028564
Iteration: 700  Train Loss: 0.18824245035648346
Iteration: 900  Train Loss: 0.3308257460594177
Iteration: 1000  Train Loss: 0.4738938808441162
Iteration: 1100  Train Loss: 0.22250866889953613
Iteration: 1200  Train Loss: 0.07503364980220795
Iteration: 1300  Train Loss: 0.16306276619434357
Iteration: 1400  Train Loss: 0.2640307545661926
Iteration: 1500  Train Loss: 0.08506952971220016
Iteration: 1600  Train Loss: 0.08784216642379761
Iteration: 1700  Train Loss: 0.1064421683549881
Iteration: 1800  Train Loss: 0.077568918466568
Iteration: 1900  Train Loss: 0.07912462204694748
Iteration: 2000  Train Loss: 0.09290442615747452
Iteration: 2100  Train Loss: 0.13587294518947